In [1]:
import cdsapi

c = cdsapi.Client()
c.retrieve(
    "reanalysis-era5-complete", 
    {
        "date": "20130101",  # The hyphens can be omitted
        "levelist": "131/to/137",  # 1 is top level, 137 the lowest model level in ERA5. Use '/' to separate values.
        "levtype": "ml",
        "param": "131/132",  # Full information at https://apps.ecmwf.int/codes/grib/param-db/
        # The native representation for temperature is spherical harmonics
        "stream": "oper",  # Denotes ERA5. Ensemble members are selected by 'enda'
        "time": "00/to/23/by/1",
        "type": "an",
        "grid": "0.25/0.25",  # Latitude/longitude. Default: spherical harmonics or reduced Gaussian grid
        "format": "netcdf",  # Output needs to be regular lat-lon, so only works in combination with 'grid'!
    },
    "sample.nc",
)

2023-09-14 12:56:24,687 INFO Welcome to the CDS
2023-09-14 12:56:24,687 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-complete
2023-09-14 12:56:24,985 INFO Request is queued
2023-09-14 12:56:26,177 INFO Request is running
2023-09-14 12:58:20,436 INFO Request is completed
2023-09-14 12:58:20,441 INFO Downloading https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data1/adaptor.mars.external-1694721466.5459745-30141-2-d77c6ff2-f206-4c7d-bdd4-ecdb53eac12b.nc to sample.nc (665.4M)
2023-09-14 13:10:20,887 INFO Download rate 945.7K/s 


Result(content_length=697707568,content_type=application/x-netcdf,location=https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data1/adaptor.mars.external-1694721466.5459745-30141-2-d77c6ff2-f206-4c7d-bdd4-ecdb53eac12b.nc)

### Jit function with parallelization

In [2]:
from numba import jit, prange

In [16]:
@jit(parallel=True)
def prange_test(A):
    s = 0
    # Without "parallel=True" in the jit-decorator
    # the prange statement is equivalent to range
    for i in prange(A.shape[0]):
        s += A[i]
    return s

/var/folders/60/xv39hkc90wn5k5b8v0f6j_v80000gn/T/ipykernel_76146/363801367.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(parallel=True)


In [17]:
import numpy as np
input_data = np.array([1, 2, 3, 4, 5])
result = prange_test(input_data)
print(result)

15


### ERA5 Testing

In [4]:
import logging
logging.basicConfig(level=logging.INFO)
import geodata
from geodata.model.wind import WindInterpolationModel

In [3]:
DS = geodata.Dataset(
    module="era5",
    weather_data_config="wind_solar_hourly",
    years=slice(2005, 2005),
    months=slice(1,1),
    bounds=[50, -3, 45, 3]
)

2023-09-18 20:02:06,702 - geodata.dataset - INFO - Directory /Users/apple/.local/geodata/era5 found, checking for completeness.
2023-09-18 20:02:06,703 - geodata.dataset - INFO - Directory complete.


In [ ]:
model = WindInterpolationModel(DS)
model.prepare()